In [39]:
# usage: given a tweet-id and retrieve the following tweet replies

In [40]:
%pwd

'/Users/binghe/Dropbox/gatech/github/aml/counter_misinformation'

In [1]:
from twarc import Twarc2, expansions
import datetime
import json
import matplotlib.pyplot as plt
import csv
import os
import sys
# Replace your bearer token below
from constant import  BEAR_CDOE_BH, BEAR_CODE_SY, BEAR_CODE_GV # BEAR_CDOE_BH
client = Twarc2(bearer_token=BEAR_CODE_SY)
from constant import TEXT, ID, REFERENCED_TWEET
from utility import tokenize

NameError: name 'tweet_look_up_by_id' is not defined

In [42]:
import pandas as pd
from constant import *

In [43]:
def tweet_look_up_by_id(tweet_ids: list, client):
    lookup = client.tweet_lookup(tweet_ids=tweet_ids)
    tweet_res = []
    try:
        for page in lookup:
            try:
                result = expansions.flatten(page)
                for tweet in result:
                    tweet_res.append(tweet)
            except ValueError:
                print(f"skip one unaccessible tweet id in one page and continue")
                continue
        if len(tweet_res) < 1:
            return None
        else:
            return tweet_res
    except ValueError:
        print(f"encounter one unaccessible tweet id in one lookup(batch)")
        return None

In [44]:
import csv
def save_tweet_and_1st_reply(tweet, replies, csv_fp):
    with open(csv_fp, "w", encoding="utf-8") as f:
        csv_writer = csv.writer(f)
        header = ["id", "tweet", "reply", "label"]
        csv_writer.writerow(header)
        for index, reply in enumerate(replies):
            if index == 0:
                # change from tweet[ID] to reply[ID]
                csv_writer.writerow([reply[ID], tweet[TEXT], reply[TEXT], ""])
            else:
                csv_writer.writerow([reply[ID], "", reply[TEXT], ""])
        print(f"we have {index} replies")

In [45]:
from utility import process_tweet
from nltk.tokenize import word_tokenize
from utility import tweet_look_up_by_id

In [46]:
if_filtering_short_tweets = False

if if_filtering_short_tweets:
    MIN_LEN_TWEET = 3 # after ieee bigdata'20 preprocessing

# condition 0: one csv with tweets in different timestamps
# when I do the initial labeling of 500 tweets, I use this code snippet.

In [48]:
# from one example after classification
# csv_fp = "./toy_example_data/misinfo_only_tweet_2021-06-26.csv"
# original tweet annotation result
# # replies: 15~50
# csv_fp = "./toy_example_data/anti_vax_misinfo_only_tweets_filtered_reply_15_to_50.csv"
# saved_convo_dir = "./toy_example_data/anti_vax_convo_15_to_50"
# # of replies: 10~15
csv_fp = "./toy_example_data/anti_vax_misinfo_only_tweets_filtered_reply_10_to_15.csv"
saved_convo_dir = "./toy_example_data/anti_vax_convo_10_to_15"

df = pd.read_csv(csv_fp)
print(df.head(5))

                    id                                               text  \
0  1342642741927432195  These rushed vaccines and their campaigns, wil...   
1  1342622908468879368  Correct me if I’m wrong, but the COVID “vaccin...   
2  1342585603825618944  Medical experts need to stop lying on behalf o...   
3  1346724774203027456  https://t.co/3thOnspvkZ\nn=2244 interviews wit...   
4  1346557934684176386  Let's sum this up: The government demands that...   

   #reply  
0      11  
1      12  
2      10  
3      10  
4      14  


In [50]:
# ===== a list of tweet ids for conversation setup ====
tweets = tweet_look_up_by_id(df["id"].tolist(), client)
# ==== only 1 tweet conversation ====
# tweets = tweet_look_up_by_id( [1402517058840039426], client)

In [51]:
# look up the whole conversation from a tweet-id
for tweet_obj in tweets:
    tweet_id = tweet_obj[ID]

    convo_csv_fp = f"{saved_convo_dir}/{tweet_id}.csv"
    #%%
    created_at = tweet_obj[CREATED_AT]
    #%%
    # print(created_at)
    #%%
    year = created_at[0:4]
    month = created_at[5:7]
    day = created_at[8:10]
    #%%
    conv_id = tweet_obj[CONVERSATION_ID]
    print(f" process tweet-id: {tweet_id}, convo-id: {conv_id}")
    start_time = datetime.datetime(int(year), int(month), int(day), 0, 0, 0, 0,
                                   datetime.timezone.utc)
    #%%
    query = f"conversation_id:{conv_id}"
    search_results = client.search_all(query=query,
                                       start_time=start_time, max_results=100)

    #%%

    referenced_id2tweet_obj = {}
    for page in search_results:
        # put the page loop first, since, we may not have the conversations
        result = expansions.flatten(page)
        for tweet in result:
            # print(json.dumps(tweet))
            # output_file.write(f"{json.dumps(tweet)}\n")
            # print(tweet)
            # print(tweet[ID])
            # ==== check the length of the tweet ====
            if if_filtering_short_tweets and len(word_tokenize(process_tweet(tweet[TEXT]))) < MIN_LEN_TWEET:
                continue
            # ==== end ====
            referenced_tweet_id = tweet[REFERENCED_TWEET][0][ID]
            if referenced_tweet_id in referenced_id2tweet_obj:
                referenced_id2tweet_obj[referenced_tweet_id].append(tweet)
            else:
                referenced_id2tweet_obj[referenced_tweet_id] = [tweet]

    save_tweet_and_1st_reply(tweet_obj,
                             referenced_id2tweet_obj[tweet_obj[ID]],
                             convo_csv_fp)
    # print(year, month, day)

 process tweet-id: 1342642741927432195, convo-id: 1342642741927432195
we have 9 replies
 process tweet-id: 1342622908468879368, convo-id: 1342622908468879368
we have 9 replies
 process tweet-id: 1342585603825618944, convo-id: 1342585603825618944
we have 2 replies
 process tweet-id: 1346724774203027456, convo-id: 1346724774203027456
we have 4 replies
 process tweet-id: 1346557934684176386, convo-id: 1346557934684176386
we have 9 replies
 process tweet-id: 1346344148874498049, convo-id: 1346344148874498049
we have 9 replies
 process tweet-id: 1346323182094057472, convo-id: 1346323182094057472
we have 9 replies
 process tweet-id: 1346107112749989888, convo-id: 1346107112749989888
we have 3 replies
 process tweet-id: 1346045122719322114, convo-id: 1346045122719322114
we have 4 replies
 process tweet-id: 1345757507319443457, convo-id: 1345757507319443457
we have 7 replies
 process tweet-id: 1344999203894865922, convo-id: 1344999203894865922
we have 10 replies
 process tweet-id: 136077273007

3


# condition 1: other cases: we have csvs in a directory and
# save the convo in year-month-day/id.csv format

In [8]:
# year = 2021
# month = 6
# day = 26

csv_dir = "/home/bhe46/data/ANTiVax/tweet_mis_only_csv"
saved_convo_dir_parent = "/home/bhe46/data/ANTiVax/tweet_mis_only_csv_then_convo"


In [9]:
for csv_fn in os.listdir(csv_dir):
    csv_fp = f"{csv_dir}/{csv_fn}"
    print(f"process file: {csv_fp}")
    year = csv_fp[-14:-10]
    month = csv_fp[-9:-7]
    day = csv_fp[-6:-4]

    convo_csv_dir = f"{saved_convo_dir_parent}/{year}-{month}-{day}"

    if not os.path.exists(convo_csv_dir):
        os.makedirs(convo_csv_dir)


    df = pd.read_csv(csv_fp)
    # ==== only get tweets with more than 3 replies ====
    df = df[ df["#reply"] >= 3 ]
    # ==== end ====
    tweets = tweet_look_up_by_id(df["id"].tolist(), client)


    for tweet_obj in tweets:
        tweet_id = tweet_obj[ID]

        convo_csv_fp = f"{convo_csv_dir}/{tweet_id}.csv"
        #%%
        created_at = tweet_obj[CREATED_AT]
        #%%
        # print(created_at)
        #%%
        year = created_at[0:4]
        month = created_at[5:7]
        day = created_at[8:10]
        #%%
        conv_id = tweet_obj[CONVERSATION_ID]
        print(f" process tweet-id: {tweet_id}, convo-id: {conv_id}")
        start_time = datetime.datetime(int(year), int(month), int(day), 0, 0, 0, 0,
                                       datetime.timezone.utc)
        #%%
        query = f"conversation_id:{conv_id}"
        search_results = client.search_all(query=query,
                                           start_time=start_time, max_results=100)

        #%%

        referenced_id2tweet_obj = {}
        for page in search_results:
            # put the page loop first, since, we may not have the conversations
            result = expansions.flatten(page)
            for tweet in result:
                # print(json.dumps(tweet))
                # output_file.write(f"{json.dumps(tweet)}\n")
                # print(tweet)
                # print(tweet[ID])
                # ==== check the length of the tweet ====
                if if_filtering_short_tweets and len(word_tokenize(process_tweet(tweet[TEXT]))) < MIN_LEN_TWEET:
                    continue
                # ==== end ====
                referenced_tweet_id = tweet[REFERENCED_TWEET][0][ID]
                if referenced_tweet_id in referenced_id2tweet_obj:
                    referenced_id2tweet_obj[referenced_tweet_id].append(tweet)
                else:
                    referenced_id2tweet_obj[referenced_tweet_id] = [tweet]

        save_tweet_and_1st_reply(tweet_obj,
                                 referenced_id2tweet_obj[tweet_obj[ID]],
                                 convo_csv_fp)
        # print(year, month, day)

process file: /home/bhe46/data/ANTiVax/tweet_mis_only_csv/2021-05-07.csv


NameError: name 'client' is not defined

# condition 2: tweet_id == convo_id

In [36]:

df = pd.read_csv(csv_fp)


In [ ]:
saved_convo_dir = f"./toy_example_data/{year}_{month}_{day}"

In [ ]:
for tweet_id in list(df[ID]):
    tweet_obj = tweet_look_up_by_id([tweet_id], client)[0]
    tweet_id = tweet_obj[ID]
    convo_csv_fp = f"{saved_convo_dir}/{tweet_id}.csv"


    conv_id = tweet_obj[CONVERSATION_ID]
    query = f"conversation_id:{conv_id}"
    search_results = client.search_all(query=query,
                                       start_time=start_time, max_results=100)

    #%%

    referenced_id2tweet_obj = {}
    for page in search_results:
        # put the page loop first, since, we may not have the conversations
        result = expansions.flatten(page)
        for tweet in result:
            # print(json.dumps(tweet))
            # output_file.write(f"{json.dumps(tweet)}\n")
            # print(tweet)
            # print(tweet[ID])
            referenced_tweet_id = tweet[REFERENCED_TWEET][0][ID]
            if referenced_tweet_id in referenced_id2tweet_obj:
                referenced_id2tweet_obj[referenced_tweet_id].append(tweet)
            else:
                referenced_id2tweet_obj[referenced_tweet_id] = [tweet]

    save_tweet_and_1st_reply(tweet_obj,
                             referenced_id2tweet_obj[tweet_obj[ID]],
                             convo_csv_fp)

# condition 2: tweet_id != convo_id, then, we only save the related first-level setup
# then, the setup is the same

# Step 2: save the file, one convo, one csv file

In [62]:
# we process it the classification step
# df = pd.read_csv("./toy_example_data/top_down_test.csv")
# df["processed_reply"] = df["reply"].apply(tokenize)
# df.to_csv("./toy_example_data/top_down_test.csv", index=False)